In [173]:
import sqlite3
from acrg.acrg_config.paths import paths
import xarray as xr
import os
import pandas as pd

In [174]:
# Directories to exclude from database
exclude = ["GOSAT", "unknown"]

network = []
instrument = []
site_code = []
start_date = []
end_date = []
species = []
height = []
calibration_scale = []
filename = []

# Find sub-directories in obs folder
for d in paths.obs.glob("*"):
    if d.is_dir():
        if d.name not in exclude:
            
            # Find netcdf files
            files = d.glob("*.nc")
            for f in files:
                
                # TODO: Some files are empty. Figure out why!
                if os.stat(f).st_size != 0:
                    
                    #TODO: add try/except to see if files open with xarray 
                    
                    f_parts = f.name.split("_")

                    network.append(f_parts[0].split("-")[0])
                    instrument.append(f_parts[0].split("-")[1])

                    site_code.append(f_parts[1])

                    extras = f_parts[3].split("-")
                    species.append(extras[0])
                    if len(extras) == 3:
                        height.append(extras[1])
                    else:
                        height.append("%")

                    with xr.open_dataset(f) as ds:
                        if "Calibration_scale" in ds.attrs.keys():
                            calibration_scale.append(ds.attrs["Calibration_scale"])
                        else:
                            calibration_scale.append(None)
                        start_date.append(pd.Timestamp(ds["time"].values[0]).to_pydatetime())
                        end_date.append(pd.Timestamp(ds["time"].values[-1]).to_pydatetime())
                            
                    filename.append(str(f))


In [175]:
file_info = list(zip(filename, network, instrument, site_code, species, height, calibration_scale, start_date, end_date))

In [176]:
# Write database

conn = sqlite3.connect(paths.obs / "obs.db")
c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS files
          ''')
c.execute('''CREATE TABLE files
             (filename text, network text, instrument text, site text, species text, height text, scale text, startDate timestamp, endDate timestamp)''')

c.executemany('INSERT INTO files VALUES (?,?,?,?,?,?,?,?,?)', file_info)

conn.commit()
conn.close()


In [171]:
# Test

conn = sqlite3.connect(paths.obs / "obs.db")

c = conn.cursor()

species_site_height = ("N2O", "bsd", "248m")
for row in c.execute('''SELECT startDate FROM files WHERE 
                        species=? COLLATE NOCASE AND
                        site=? COLLATE NOCASE AND
                        height=?
                        ''', species_site_height):
    print(row)

conn.close()

('2017-03-17 16:55:22.500000',)
('2019-03-06 13:23:30',)


In [172]:
# Test pandas

conn = sqlite3.connect(paths.obs / "obs.db")

df = pd.read_sql_query("SELECT * FROM files", conn, parse_dates=["startDate", "endDate"])

conn.close()